# Decission Tree Regression Workflow

This file demonstrates the workflow used for optimizing, training, and evaluating the decission tree regression models trained for predicting the soft phonon frequency of KTaO3. Start by loading the data sets.

In [1]:
import pandas as pd

df = pd.read_csv('data/')
 



/Users/zachv/Documents/research-github/kto-phonon-study
